In [1]:
import os
import utils as u

In [19]:
list1_path='flux/2017' # Raw local #flux 2017 hors ETS
list2_path='data/prep_flux_2017_horsETS' # Prep local #flux 2017 hors ETS
list3_path='INPI/TC_1/Flux/2018' #raw like S3
list4_path='INPI/TC_1/01_donnee_source/Flux/2018/ETS' #prepared S3
list5_path='INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2019/ETS' #raw S3

# List 1 : raw local

In [ ]:

list_files_raw_local=[]
for root,dirs,files in os.walk(list1_path):
    for file in files:
          if file.endswith(('.csv')):
                type=file.split('_')[4] 
                if (type not in ['8','9','10']):# EXCLUDE ETS NEW ET EVT
                    list_files_raw_local.append(str(file))
list_files_raw_local[:5]

# List 2 : prepared local

In [ ]:
list_files_prep_local=[]
for root,dirs,files in os.walk(list2_path):
    for file in files:
          if file.endswith(('.csv')):
                #print(file)
                list_files_prep_local.append(str(file))
list_files_prep_local[:5]

In [ ]:
diff_list=list(set(list_files_raw_local) - set(list_files_prep_local))
diff_list=sorted(diff_list)
diff_list[:100]

# List Files S3

In [15]:
    # Import S3 connectors librairies
    #from awsPy.aws_authorization import aws_connector
    import importlib.util
    spec = importlib.util.spec_from_file_location("aws_connector.py", "../../../../aws-python/awsPy/aws_authorization/aws_connector.py")
    aws_connector = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(aws_connector)

    #from awsPy.aws_s3 import service_s3
    import importlib.util
    spec = importlib.util.spec_from_file_location("service_s3.py", "../../../../aws-python/awsPy/aws_s3/service_s3.py")
    service_s3 = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(service_s3)

    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')
    bucket_name='calfdata'
    bucket = client['resource'].Bucket(bucket_name)

In [16]:
list_files_rawlike_s3=s3.list_files_s3(list3_path)
tmp=[]
for i in list_files_rawlike_s3:
    if i.endswith('.csv'):
        filename=i.split('/')[-1]
        tmp.append(filename)
list_files_rawlike_s3=tmp
list_files_rawlike_s3[:5]

KeyboardInterrupt: 

In [17]:
list_files_prep_s3=s3.list_files_s3(list4_path)
tmp=[]
for i in list_files_prep_s3:
    if i.endswith('.csv'):
        filename=i.split('/')[-1]
        tmp.append(filename)
list_files_prep_s3=tmp
list_files_prep_s3[:5]

['0101_228_20180407_230513_10_ets_supprime_EVT.csv',
 '0101_228_20180407_230513_9_ets_nouveau_modifie_EVT.csv',
 '0101_229_20180408_085432_10_ets_supprime_EVT.csv',
 '0101_229_20180408_085432_9_ets_nouveau_modifie_EVT.csv',
 '0101_230_20180408_105618_10_ets_supprime_EVT.csv']

In [20]:
list_files_raw_s3=s3.list_files_s3(list5_path)
tmp=[]
for i in list_files_raw_s3:
    if i.endswith('.csv'):
        filename=i.split('/')[-1]
        tmp.append(filename)
list_files_raw_s3=tmp
list_files_raw_s3[:5]

[]

In [7]:
diff_list_s3=list(set(list_files_raw_s3) - set(list_files_prep_s3))
diff_list_s3=sorted(diff_list_s3)
diff_list_s3[:10]

['0101_27_20170616_071416_1_PM.csv',
 '0101_28_20170617_071244_1_PM.csv',
 '0101_33_20170624_075849_1_PM.csv',
 '0101_67_20170819_071109_6_rep_nouveau_modifie_EVT.csv',
 '0203_19_20170602_092437_7_rep_partant_EVT.csv',
 '0301_54_20170731_180136_9_ets_nouveau_modifie_EVT.csv',
 '0303_44_20170711_091821_9_ets_nouveau_modifie_EVT.csv',
 '0603_4_20170512_121457_5_rep.csv',
 '0605_282_20171130_083546_1_PM.csv',
 '0605_284_20171202_063246_1_PM.csv']

In [8]:
len(list_files_raw_s3)

308074

In [9]:
len(list_files_rawlike_s3)

323280

In [18]:
len(list_files_prep_s3)

95360

In [11]:
len(diff_list_s3)

256

In [12]:
len(list_files_raw_s3)-len(list_files_prep_s3)

256

In [13]:
len(list_files_raw_s3)-len(list_files_rawlike_s3)

-15206

# List files FTP

In [ ]:
# INPI FTP Connexion parameters
host = 'opendata-rncs.inpi.fr'
port = 21
usr = 'calf'
pwd = 'Calfinpi2019'
path_local = "/home/ec2-user/SageMaker/Extract_inpi_FTS/temp"

In [ ]:
import os
import ftplib
import ftputil
import ftputil.session
#https://ftputil.sschwarzer.net/trac/wiki/Documentation

# Enable secure session
my_session_factory = ftputil.session.session_factory(
                       base_class=ftplib.FTP_TLS,
                       port=port,
                       encrypt_data_channel=True)

# create FTP Host
ftp_host = ftputil.FTPHost(host, usr, pwd,session_factory=my_session_factory)

In [ ]:
# List files in FTP Directory
path='/public/IMR_Donnees_Saisies/tc/flux/2018'

list_files_ftp=[]
for root,dirs,files in ftp_host.walk(path):
    for file in files:
          if file.endswith(('.csv','.zip')):
                list_files_ftp.append((root, file))
list_files_ftp[:5]

In [ ]:
print (str(len(list_files_ftp)) + ' files in ' + path)